In [1]:
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

In [2]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 16192775716432439225, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 9214062756
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 2586003734976851017
 physical_device_desc: "device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1"]

In [3]:
import sys
import time
import math
import os
import pandas as pd
import numpy as np

import keras
from keras import layers, metrics, models, optimizers
from keras.optimizers import Adam, RMSprop,SGD
from keras.models import Model
from keras.models import Sequential
from keras.layers import Input, concatenate, Flatten, Dense, Activation, Dropout, Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator


import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [4]:
import keras
import tensorflow as tf

print("Keras Version", keras.__version__)
print("tensorflow Version", tf.__version__)
# print("dim_ordering:", K.image_dim_ordering())

Keras Version 2.2.4
tensorflow Version 1.9.0


In [5]:
batch_size = 16
img_rows, img_cols = 256, 256
input_shape = (img_rows, img_cols, 3)
epochs = 150

In [6]:
train_dir = 'data_augmented/cvc300_train/'
validation_dir = 'data_augmented/cvc300_validation/'
test_dir = 'data_augmented/cvc300_test/'

image_folder = 'images'
masks_folder = 'masks'

In [7]:
# -*- coding: utf-8 -*-
from keras.applications.vgg16 import VGG16
from keras.layers import Input
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.core import Dropout, Lambda
from keras.layers.merge import concatenate
from keras.models import Model


def Vgg16Unet(img_size):
    inputs = Input(shape=(img_size, img_size, 3))
    s = Lambda(lambda x: x / 255)(inputs)
    vgg16 = VGG16(include_top=False, weights='imagenet',
                  input_tensor=s)
    for layer in vgg16.layers:
        layer.trainable = False

    c1 = vgg16.get_layer("block1_conv2").output
    c2 = vgg16.get_layer("block2_conv2").output
    c3 = vgg16.get_layer("block3_conv3").output
    c4 = vgg16.get_layer("block4_conv3").output
    c5 = vgg16.get_layer("block5_conv3").output

    u6 = Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = concatenate([u6, c4])
    c6 = Conv2D(512, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(u6)
    c6 = Dropout(0.2)(c6)
    c6 = Conv2D(512, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(c6)

    u7 = Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = concatenate([u7, c3])
    c7 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(u7)
    c7 = Dropout(0.2)(c7)
    c7 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(c7)

    u8 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c7)
    u8 = concatenate([u8, c2])
    c8 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(u8)
    c8 = Dropout(0.1)(c8)
    c8 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(c8)

    u9 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c8)
    u9 = concatenate([u9, c1], axis=3)
    c9 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(u9)
    c9 = Dropout(0.1)(c9)
    c9 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(c9)

    outputs = Conv2D(1, (1, 1), activation='sigmoid')(c9)

    model = Model(inputs=[vgg16.input], outputs=[outputs])

    return model


In [8]:
seed = 1142
data_gen_args = dict(rescale=1. / 255, featurewise_center=True, featurewise_std_normalization=True)
image_datagen = ImageDataGenerator(**data_gen_args)
masks_datagen = ImageDataGenerator(**data_gen_args)

test_datagen = ImageDataGenerator(rescale=1./255)

image_generator = image_datagen.flow_from_directory(
    train_dir,
    classes = [image_folder],
    target_size=(img_rows, img_cols),
    batch_size=batch_size,
    class_mode=None,
    seed=seed)

mask_generator = masks_datagen.flow_from_directory(
    train_dir,
    classes = [masks_folder],
    color_mode='grayscale',
    target_size=(img_rows, img_cols),
    batch_size=batch_size,
    class_mode=None,
    seed=seed)

validation_image_generator = image_datagen.flow_from_directory(
    validation_dir,
    classes = [image_folder],
    target_size=(img_rows, img_cols),
    batch_size=batch_size,
    class_mode=None,
    seed=seed)

validation_mask_generator = masks_datagen.flow_from_directory(
    validation_dir,
    classes = [masks_folder],
    color_mode='grayscale',
    target_size=(img_rows, img_cols),
    batch_size=batch_size,
    class_mode=None,
    seed=seed)

test_image_generator = test_datagen.flow_from_directory(
    test_dir,
    classes = [image_folder],
    target_size=(img_rows, img_cols),
    batch_size=batch_size,
    class_mode=None,
    seed=seed)

test_mask_generator = test_datagen.flow_from_directory(
    test_dir,
    classes = [masks_folder],
    color_mode='grayscale',
    target_size=(img_rows, img_cols),
    batch_size=batch_size,
    class_mode=None,
    seed=seed)

Found 26112 images belonging to 1 classes.
Found 26112 images belonging to 1 classes.
Found 8704 images belonging to 1 classes.
Found 8704 images belonging to 1 classes.
Found 28 images belonging to 1 classes.
Found 28 images belonging to 1 classes.


In [9]:
train_generator = zip(image_generator, mask_generator)
validation_generator = zip(validation_image_generator, validation_mask_generator)
test_generator = zip(test_image_generator, test_mask_generator)

In [10]:
nb_train_samples = len(image_generator.filenames)
nb_validation_samples = len(validation_image_generator.filenames)
nb_test_samples = len(test_image_generator.filenames)

predict_size_train = int(math.ceil(nb_train_samples / batch_size))
predict_size_validation = int(math.ceil(nb_validation_samples / batch_size))
predict_size_test = int(math.ceil(nb_test_samples / batch_size))

print("nb_train_samples:", nb_train_samples)
print("nb_validation_samples:", nb_validation_samples)
print("nb_test_samples:", nb_test_samples)

print("\npredict_size_train:", predict_size_train)
print("predict_size_validation:", predict_size_validation)
print("predict_size_test:", predict_size_test)

nb_train_samples: 26112
nb_validation_samples: 8704
nb_test_samples: 28

predict_size_train: 1632
predict_size_validation: 544
predict_size_test: 2


In [11]:
smooth = 1.
from keras import backend as K

def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)


def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)


def Specificity(y_true, y_pred):
    true_negatives = K.abs(y_pred)- K.abs(y_true)
    return ((true_negatives+smooth)/(y_pred+ smooth))

def Sensitivity(y_true, y_pred):
    y_true = K.flatten(y_true)
    y_pred = K.flatten(y_pred)
    return ((y_pred+smooth)/ (y_true+smooth))

def Jaccard_index(y_true,y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return ((intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) - intersection + smooth))

def precision(y_true, y_pred):
    """Precision metric.
    Only computes a batch-wise average of precision.
    Computes the precision, a metric for multi-label classification of
    how many selected items are relevant.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision


def recall(y_true, y_pred):
    """Recall metric.
    Only computes a batch-wise average of recall.
    Computes the recall, a metric for multi-label classification of
    how many relevant items are selected.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall


def f1score(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.
        Only computes a batch-wise average of recall.
        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.
        Only computes a batch-wise average of precision.
        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2 * ((precision * recall) / (precision + recall))

In [12]:
model = Vgg16Unet(img_size = 256)
model.summary()
adam_opt = Adam(lr=1e-5, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

model.compile(optimizer= adam_opt, loss=dice_coef_loss, 
              metrics=[dice_coef, 'acc','mse', Jaccard_index, Specificity, Sensitivity, precision, recall, f1score])

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 256, 256, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 256, 256, 64) 1792        lambda_1[0][0]                   
__________________________________________________________________________________________________
block1_conv2 (Conv2D)           (None, 256, 256, 64) 36928       block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_poo

In [13]:
history = model.fit_generator(train_generator,
                    steps_per_epoch=nb_train_samples//batch_size,
                    epochs=epochs,
                    validation_data = validation_generator,
                    validation_steps = nb_validation_samples//batch_size)

Epoch 1/150
1632/1632 [==============================] - 681s 417ms/step - loss: -0.1483 - dice_coef: 0.1483 - acc: 0.5893 - mean_squared_error: 0.2808 - Jaccard_index: 0.0806 - Specificity: 0.9663 - Sensitivity: 1.2668 - precision: 0.0981 - recall: 0.5459 - f1score: 0.1618 - val_loss: -0.1857 - val_dice_coef: 0.1857 - val_acc: 0.5751 - val_mean_squared_error: 0.3065 - val_Jaccard_index: 0.1031 - val_Specificity: 0.9612 - val_Sensitivity: 1.2803 - val_precision: 0.1224 - val_recall: 0.6420 - val_f1score: 0.2044
Epoch 2/150
1632/1632 [==============================] - 663s 406ms/step - loss: -0.1604 - dice_coef: 0.1604 - acc: 0.6735 - mean_squared_error: 0.2035 - Jaccard_index: 0.0877 - Specificity: 0.9634 - Sensitivity: 1.1688 - precision: 0.1136 - recall: 0.4367 - f1score: 0.1762 - val_loss: -0.1934 - val_dice_coef: 0.1934 - val_acc: 0.6219 - val_mean_squared_error: 0.2556 - val_Jaccard_index: 0.1079 - val_Specificity: 0.9590 - val_Sensitivity: 1.2224 - val_precision: 0.1317 - val_rec

Epoch 17/150
1632/1632 [==============================] - 641s 393ms/step - loss: -0.1664 - dice_coef: 0.1664 - acc: 0.7265 - mean_squared_error: 0.1465 - Jaccard_index: 0.0915 - Specificity: 0.9610 - Sensitivity: 1.1035 - precision: 0.1268 - recall: 0.3351 - f1score: 0.1823 - val_loss: -0.1992 - val_dice_coef: 0.1992 - val_acc: 0.7075 - val_mean_squared_error: 0.1634 - val_Jaccard_index: 0.1120 - val_Specificity: 0.9539 - val_Sensitivity: 1.1148 - val_precision: 0.1534 - val_recall: 0.3816 - val_f1score: 0.2168
Epoch 18/150
1632/1632 [==============================] - 641s 393ms/step - loss: -0.1663 - dice_coef: 0.1663 - acc: 0.7267 - mean_squared_error: 0.1462 - Jaccard_index: 0.0914 - Specificity: 0.9610 - Sensitivity: 1.1032 - precision: 0.1268 - recall: 0.3340 - f1score: 0.1822 - val_loss: -0.1984 - val_dice_coef: 0.1984 - val_acc: 0.7122 - val_mean_squared_error: 0.1585 - val_Jaccard_index: 0.1116 - val_Specificity: 0.9535 - val_Sensitivity: 1.1088 - val_precision: 0.1547 - val_r

1632/1632 [==============================] - 641s 393ms/step - loss: -2.1187e-05 - dice_coef: 2.1187e-05 - acc: 0.8261 - mean_squared_error: 0.0408 - Jaccard_index: 2.1187e-05 - Specificity: 0.9533 - Sensitivity: 0.9748 - precision: 0.0000e+00 - recall: 0.0000e+00 - f1score: nan - val_loss: -1.7611e-05 - val_dice_coef: 1.7611e-05 - val_acc: 0.8176 - val_mean_squared_error: 0.0500 - val_Jaccard_index: 1.7611e-05 - val_Specificity: 0.9431 - val_Sensitivity: 0.9693 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_f1score: nan
Epoch 33/150
1632/1632 [==============================] - 641s 393ms/step - loss: -2.1109e-05 - dice_coef: 2.1109e-05 - acc: 0.8261 - mean_squared_error: 0.0408 - Jaccard_index: 2.1109e-05 - Specificity: 0.9533 - Sensitivity: 0.9748 - precision: 0.0000e+00 - recall: 0.0000e+00 - f1score: nan - val_loss: -1.7580e-05 - val_dice_coef: 1.7580e-05 - val_acc: 0.8176 - val_mean_squared_error: 0.0500 - val_Jaccard_index: 1.7580e-05 - val_Specificity: 0.9431 - val_S

Epoch 47/150
1632/1632 [==============================] - 641s 393ms/step - loss: -2.1144e-05 - dice_coef: 2.1144e-05 - acc: 0.8261 - mean_squared_error: 0.0408 - Jaccard_index: 2.1144e-05 - Specificity: 0.9533 - Sensitivity: 0.9748 - precision: 0.0000e+00 - recall: 0.0000e+00 - f1score: nan - val_loss: -1.7602e-05 - val_dice_coef: 1.7602e-05 - val_acc: 0.8176 - val_mean_squared_error: 0.0500 - val_Jaccard_index: 1.7602e-05 - val_Specificity: 0.9431 - val_Sensitivity: 0.9693 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_f1score: nan
Epoch 48/150
1632/1632 [==============================] - 642s 393ms/step - loss: -2.1164e-05 - dice_coef: 2.1164e-05 - acc: 0.8261 - mean_squared_error: 0.0408 - Jaccard_index: 2.1164e-05 - Specificity: 0.9533 - Sensitivity: 0.9748 - precision: 0.0000e+00 - recall: 0.0000e+00 - f1score: nan - val_loss: -1.7571e-05 - val_dice_coef: 1.7571e-05 - val_acc: 0.8176 - val_mean_squared_error: 0.0500 - val_Jaccard_index: 1.7571e-05 - val_Specificity: 0

Epoch 62/150
1632/1632 [==============================] - 641s 393ms/step - loss: -2.1140e-05 - dice_coef: 2.1140e-05 - acc: 0.8261 - mean_squared_error: 0.0408 - Jaccard_index: 2.1140e-05 - Specificity: 0.9533 - Sensitivity: 0.9748 - precision: 0.0000e+00 - recall: 0.0000e+00 - f1score: nan - val_loss: -1.7592e-05 - val_dice_coef: 1.7592e-05 - val_acc: 0.8176 - val_mean_squared_error: 0.0500 - val_Jaccard_index: 1.7592e-05 - val_Specificity: 0.9431 - val_Sensitivity: 0.9693 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_f1score: nan
Epoch 63/150
1632/1632 [==============================] - 641s 393ms/step - loss: -2.1138e-05 - dice_coef: 2.1138e-05 - acc: 0.8261 - mean_squared_error: 0.0408 - Jaccard_index: 2.1138e-05 - Specificity: 0.9533 - Sensitivity: 0.9748 - precision: 0.0000e+00 - recall: 0.0000e+00 - f1score: nan - val_loss: -1.7602e-05 - val_dice_coef: 1.7602e-05 - val_acc: 0.8176 - val_mean_squared_error: 0.0500 - val_Jaccard_index: 1.7602e-05 - val_Specificity: 0

Epoch 77/150
1632/1632 [==============================] - 642s 393ms/step - loss: -2.1175e-05 - dice_coef: 2.1175e-05 - acc: 0.8261 - mean_squared_error: 0.0408 - Jaccard_index: 2.1175e-05 - Specificity: 0.9533 - Sensitivity: 0.9748 - precision: 0.0000e+00 - recall: 0.0000e+00 - f1score: nan - val_loss: -1.7626e-05 - val_dice_coef: 1.7626e-05 - val_acc: 0.8176 - val_mean_squared_error: 0.0500 - val_Jaccard_index: 1.7626e-05 - val_Specificity: 0.9431 - val_Sensitivity: 0.9693 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_f1score: nan
Epoch 78/150
1632/1632 [==============================] - 642s 393ms/step - loss: -2.1183e-05 - dice_coef: 2.1183e-05 - acc: 0.8261 - mean_squared_error: 0.0408 - Jaccard_index: 2.1183e-05 - Specificity: 0.9533 - Sensitivity: 0.9748 - precision: 0.0000e+00 - recall: 0.0000e+00 - f1score: nan - val_loss: -1.7612e-05 - val_dice_coef: 1.7612e-05 - val_acc: 0.8176 - val_mean_squared_error: 0.0500 - val_Jaccard_index: 1.7612e-05 - val_Specificity: 0

Epoch 92/150
1632/1632 [==============================] - 645s 395ms/step - loss: -2.1185e-05 - dice_coef: 2.1185e-05 - acc: 0.8261 - mean_squared_error: 0.0408 - Jaccard_index: 2.1185e-05 - Specificity: 0.9533 - Sensitivity: 0.9748 - precision: 0.0000e+00 - recall: 0.0000e+00 - f1score: nan - val_loss: -1.7547e-05 - val_dice_coef: 1.7547e-05 - val_acc: 0.8176 - val_mean_squared_error: 0.0500 - val_Jaccard_index: 1.7547e-05 - val_Specificity: 0.9431 - val_Sensitivity: 0.9693 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_f1score: nan
Epoch 93/150
1632/1632 [==============================] - 644s 394ms/step - loss: -2.1176e-05 - dice_coef: 2.1176e-05 - acc: 0.8261 - mean_squared_error: 0.0408 - Jaccard_index: 2.1176e-05 - Specificity: 0.9533 - Sensitivity: 0.9748 - precision: 0.0000e+00 - recall: 0.0000e+00 - f1score: nan - val_loss: -1.7496e-05 - val_dice_coef: 1.7496e-05 - val_acc: 0.8176 - val_mean_squared_error: 0.0500 - val_Jaccard_index: 1.7496e-05 - val_Specificity: 0

Epoch 107/150
1632/1632 [==============================] - 642s 394ms/step - loss: -2.1138e-05 - dice_coef: 2.1138e-05 - acc: 0.8261 - mean_squared_error: 0.0408 - Jaccard_index: 2.1138e-05 - Specificity: 0.9533 - Sensitivity: 0.9748 - precision: 0.0000e+00 - recall: 0.0000e+00 - f1score: nan - val_loss: -1.7597e-05 - val_dice_coef: 1.7597e-05 - val_acc: 0.8176 - val_mean_squared_error: 0.0500 - val_Jaccard_index: 1.7597e-05 - val_Specificity: 0.9431 - val_Sensitivity: 0.9693 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_f1score: nan
Epoch 108/150
1632/1632 [==============================] - 644s 395ms/step - loss: -2.1163e-05 - dice_coef: 2.1163e-05 - acc: 0.8261 - mean_squared_error: 0.0408 - Jaccard_index: 2.1163e-05 - Specificity: 0.9533 - Sensitivity: 0.9748 - precision: 0.0000e+00 - recall: 0.0000e+00 - f1score: nan - val_loss: -1.7639e-05 - val_dice_coef: 1.7639e-05 - val_acc: 0.8176 - val_mean_squared_error: 0.0500 - val_Jaccard_index: 1.7639e-05 - val_Specificity:

Epoch 122/150
1632/1632 [==============================] - 644s 394ms/step - loss: -2.1115e-05 - dice_coef: 2.1115e-05 - acc: 0.8261 - mean_squared_error: 0.0408 - Jaccard_index: 2.1115e-05 - Specificity: 0.9533 - Sensitivity: 0.9748 - precision: 0.0000e+00 - recall: 0.0000e+00 - f1score: nan - val_loss: -1.7599e-05 - val_dice_coef: 1.7599e-05 - val_acc: 0.8176 - val_mean_squared_error: 0.0500 - val_Jaccard_index: 1.7599e-05 - val_Specificity: 0.9431 - val_Sensitivity: 0.9693 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_f1score: nan
Epoch 123/150
1632/1632 [==============================] - 643s 394ms/step - loss: -2.1115e-05 - dice_coef: 2.1115e-05 - acc: 0.8261 - mean_squared_error: 0.0408 - Jaccard_index: 2.1115e-05 - Specificity: 0.9533 - Sensitivity: 0.9748 - precision: 0.0000e+00 - recall: 0.0000e+00 - f1score: nan - val_loss: -1.7596e-05 - val_dice_coef: 1.7596e-05 - val_acc: 0.8176 - val_mean_squared_error: 0.0500 - val_Jaccard_index: 1.7596e-05 - val_Specificity:

Epoch 137/150
1632/1632 [==============================] - 645s 395ms/step - loss: -2.1175e-05 - dice_coef: 2.1175e-05 - acc: 0.8261 - mean_squared_error: 0.0408 - Jaccard_index: 2.1175e-05 - Specificity: 0.9533 - Sensitivity: 0.9748 - precision: 0.0000e+00 - recall: 0.0000e+00 - f1score: nan - val_loss: -1.7535e-05 - val_dice_coef: 1.7535e-05 - val_acc: 0.8176 - val_mean_squared_error: 0.0500 - val_Jaccard_index: 1.7535e-05 - val_Specificity: 0.9431 - val_Sensitivity: 0.9693 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_f1score: nan
Epoch 138/150
1632/1632 [==============================] - 644s 394ms/step - loss: -2.1162e-05 - dice_coef: 2.1162e-05 - acc: 0.8261 - mean_squared_error: 0.0408 - Jaccard_index: 2.1162e-05 - Specificity: 0.9533 - Sensitivity: 0.9748 - precision: 0.0000e+00 - recall: 0.0000e+00 - f1score: nan - val_loss: -1.7698e-05 - val_dice_coef: 1.7698e-05 - val_acc: 0.8176 - val_mean_squared_error: 0.0500 - val_Jaccard_index: 1.7698e-05 - val_Specificity:

In [14]:
score = model.evaluate_generator(validation_generator, steps=50)

print ('Validation Score: ', score[0])
print ('Validation Accuracy: ',score[1])

Validation Score:  -1.7206792072101963e-05
Validation Accuracy:  1.7206792072101963e-05


In [15]:
prediction = model.predict_generator(test_generator, verbose=1, steps=nb_test_samples)

28/28 [==============================] - 5s 163ms/step


In [16]:
prediction.shape

(392, 256, 256, 1)

In [17]:
test_results = model.evaluate_generator(test_generator, steps=50)

print(model.metrics_names)
print(test_results)

['loss', 'dice_coef', 'acc', 'mean_squared_error', 'Jaccard_index', 'Specificity', 'Sensitivity', 'precision', 'recall', 'f1score']
[-2.251913081896159e-05, 2.2519130881326938e-05, 0.9502643043654305, 0.04973569589001792, 2.2519130881326938e-05, 0.9502643043654305, 0.9751321516718183, 0.0, 0.0, nan]


In [18]:
coef = np.array(history.history['Jaccard_index'])
val_coef = np.array(history.history['val_Jaccard_index'])
print("Training co-effiency    : {};\nValidation co-effiency : {}".format(coef[coef==max(coef)][0], val_coef[np.argmax(coef)]))

Training co-effiency    : 0.09158190552318725;
Validation co-effiency : 0.11208780340316213


In [19]:
import skimage.io as io

def labelVisualize(num_class,color_dict,img):
    img = img[:,:,0] if len(img.shape) == 3 else img
    img_out = np.zeros(img.shape + (3,))
    for i in range(num_class):
        img_out[img == i,:] = color_dict[i]
    return img_out / 255

def saveResult(save_path,npyfile,flag_multi_class = False,num_class = 2):
    for i,item in enumerate(npyfile):
        img = labelVisualize(num_class,COLOR_DICT,item) if flag_multi_class else item[:,:,0]
        io.imsave(os.path.join(save_path,"%d_predict.png"%i),img)

os.mkdir('05.U-Net2_results_dir')

FileExistsError: [WinError 183] Cannot create a file when that file already exists: '05.U-Net2_results_dir'

In [ ]:
saveResult('05.U-Net2_results_dir',prediction)

In [ ]:
def normalizeData(img,mask):
    mean = np.mean(img)
    std = np.std(img)
    img -= mean
    img /= std
    mask = mask /255
    mask[mask > 0.5] = 1
    mask[mask <= 0.5] = 0
    return (img,mask)

test_data_dir= "data_augmented/cvc300_test/images/"
test_data_mask_dir = "data_augmented/cvc300_test/masks/"
test_data_pred_dir = "05.U-Net2_results_dir"
img_rows, img_cols = 256, 256

In [ ]:
from keras.preprocessing import image as image_utils
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import array_to_img

file_names = next(os.walk(test_data_dir))[2]
scores = []
for file in file_names:
    grey_img = load_img(os.path.join(test_data_dir,file), target_size=(img_rows, img_cols), grayscale=False)
    mask_img = load_img(os.path.join(test_data_mask_dir,file), target_size=(img_rows, img_cols), grayscale=True)
    img = img_to_array(grey_img)
    img_mask = img_to_array(mask_img)
    
    img, img_mask = normalizeData(img, img_mask)
    img = np.reshape(img,(1,)+img.shape)
    
    pred = model.predict([img])
    sess = tf.Session()
    score = sess.run(Jaccard_index(img_mask, pred))
    print("{} -- jaccard index: {}".format(file,score))
    scores.append([file,score])

    result_img = array_to_img(pred[0] * 255 )
    result_img.save(os.path.join(test_data_pred_dir, file.split('.')[0] + '_predict.jpg'))

with open("unet_test_result.csv", 'w') as f:
    f.write("filename, jaccard_index\n")
    for i in range(len(scores)):
        #print(scores[i])
        f.write("{},{}\n".format(scores[i][0], scores[i][1]))

In [ ]:
%pylab inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
img=mpimg.imread('05.U-Net2_results_dir/102_predict.png')
imgplot = plt.imshow(img)
plt.show()

In [ ]:
plt.imshow(np.squeeze(prediction[1]), cmap = "gray")

In [ ]:
plt.imshow(np.squeeze(prediction[1]), cmap = "gray", vmin = 0.0, vmax= 0.09)

In [ ]:
results_ =  np.squeeze(np.multiply(prediction[1], 255))
print(results_)

plt.imshow(results_, cmap = "gray", vmin = 0.0, vmax= 0.7)
plt.show()

In [ ]:
%pylab inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
img=mpimg.imread('05.U-Net2_results_dir/35_predict.png')
imgplot = plt.imshow(img)
plt.show()

In [ ]:
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['dice_coef'])
plt.plot(history.history['val_dice_coef'])
plt.title('model dice_coeff')
plt.ylabel('dice_coef')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()